In [1]:
import os 

os.chdir("..")

In [2]:
import pandas as pd
import numpy as np

from CreditCard.exception import AppException
from CreditCard.logging import logger , get_current_time_stamp
from CreditCard.Database import MongoDB
from CreditCard.utils import load_object , read_yaml , save_object, evaluate_classification_model
from CreditCard.entity import MetricReportArtifact , ModelEvaluationArtifact , ModelEvaluationConfig
import os
import sys
from pathlib import Path



class ModelEvaluation:

    def __init__(self , model_evaluation_config : ModelEvaluationConfig):
        try:
            logger.info(f"{'>>' * 30}Model Evaluation log started.{'<<' * 30} ")
            self.model_evaluation_config = model_evaluation_config
            train_collections = self.model_evaluation_config.data_validated_train_collection
            train_connection = MongoDB(train_collections, drop_collection=False)
            self.train_df = train_connection.find_many_as_df()
            test_collections = self.model_evaluation_config.data_validated_test_collection
            test_connection = MongoDB(test_collections, drop_collection=False)
            self.test_df = test_connection.find_many_as_df()
            self.schema = read_yaml(path_to_yaml = Path(self.model_evaluation_config.schema_file_path ))
        except Exception as e:
            raise AppException(e, sys) from e
        
    def get_model_list(self , trained_model_dir_path):
        model_list = [load_object(file_path= Path(os.path.join(trained_model_dir_path , model_file))) for model_file in os.listdir(trained_model_dir_path)]
        return model_list
        

    def initiate_model_evaluation(self) -> ModelEvaluationArtifact:
        try:
            trained_model_dir = self.model_trainer_artifact.trained_model_dir
            
            model_list = self.get_model_list(trained_model_dir)
            target_column_name= self.schema.target_column
            x_train = self.train_df.drop(columns=[target_column_name ], axis=1)
            y_train = self.train_df[target_column_name]
            x_test = self.test_df.drop(columns=[target_column_name], axis=1)
            y_test = self.test_df[target_column_name]
            
            report_dir = self.model_evaluation_config.report_dir
            base_accuracy = self.model_evaluation_config.base_accuracy
            eval_difference = self.model_evaluation_config.eval_difference
            eval_param = self.model_evaluation_config.eval_param
            
            metric_report_artifact : MetricReportArtifact = evaluate_classification_model( x_train_eval= x_train , 
                                                                    y_train= y_train,
                                                                    x_test_eval = x_test,
                                                                    y_test =y_test,
                                                                    base_accuracy = base_accuracy,
                                                                    report_dir = report_dir ,
                                                                    eval_difference =eval_difference,
                                                                    estimators = model_list,
                                                                    eval_param = eval_param,
                                                                    experiment_id= "model_eval")
            
            logger.info(f"Model evaluation completed. model metric artifact: {metric_report_artifact}")

            if metric_report_artifact.model_obj  is  None:
                response = ModelEvaluationArtifact(is_model_accepted=False,
                                                   evaluated_model_path=None
                                                   )
                logger.info(response)
                return response

            else:
                logger.info("Trained model is found")
                eval_model_dir = self.model_evaluation_config.eval_model_dir
                model_file_name = f"{metric_report_artifact.model_name}.pkl"
                eval_model_path = os.path.join(eval_model_dir, model_file_name)
                save_object(obj=metric_report_artifact.model_obj ,file_path=Path(eval_model_path))
                
                model_evaluation_artifact = ModelEvaluationArtifact(evaluated_model_path=eval_model_path,
                                                                    is_model_accepted=False)
            return model_evaluation_artifact
        except Exception as e:
            logger.error(e)
            raise AppException(e, sys) from e

    def __del__(self):
        logger.info(f"{'=' * 20}Model Evaluation log completed.{'=' * 20} ")

ImportError: cannot import name 'MetricEvalArtifact' from 'CreditCard.entity.artifact_entity' (C:\Users\princ\OneDrive\Desktop\project-python\creditcard\src\CreditCard\entity\artifact_entity.py)